# Testing on caltech101 Dataset

**Importing the libraries and loading the dataset**

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from Classifier import Classifier
import os
import openai
import configparser
import torch
import clip
from torch.utils.data import Subset
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['openai']['api_key']
openai.api_key = api_key

# Load the dataset using ImageFolder
dataset = ImageFolder(root='./caltech101')
labels=dataset.classes

In [ ]:
print(f"Label: {labels[dataset[2051][1]]}")
plt.imshow(dataset[2051][0])

**Creating a classifier and loading descriptors that were generated previously**

In [ ]:
caltech=Classifier(labels)

In [ ]:
caltech.load_classifier('caltech.txt')

In [ ]:
caltech.descriptors['Motorbikes']

**Classifying with the descriptors**

indices = torch.randperm(len(dataset))[:100]
dataset2=[]
for idx in indices:
    dataset2.append(dataset[idx])

images=torch.stack([caltech.preprocess(image) for (image,label) in dataset[]])

pred=caltech.multi_classify(images,preprocessed=True)
print(f"Accuracy: {torch.mean((pred==torch.Tensor([label for (image,label) in dataset2]))+0.)}")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)
tokenized_text = torch.cat([clip.tokenize(label) for label in labels]).to(device)

In [ ]:
from time import time
sum1=0
sum2=0
for i in range(0,len(dataset),100):
    if i==8900:
        subset=Subset(dataset,[j for j in range(i,len(dataset))])
        images=torch.stack([caltech.preprocess(image) for (image,label) in subset])
        lab=torch.Tensor([label for (image,label) in subset])
    else:
        subset=Subset(dataset,[j for j in range(i,i+100)])
        images=torch.stack([caltech.preprocess(image) for (image,label) in subset])
        lab=torch.Tensor([label for (image,label) in subset])
    print(f'Starting predictions: {i}')
    t=time()
    pred1=caltech.multi_classify(images,preprocessed=True)
    sum1+=torch.sum((pred1==lab)+0.)
    
    logits=model(images,tokenized_text)[0]
    pred2=torch.argmax(logits,dim=1)
    sum2+=torch.sum((pred2==lab)+0.)
    print(time()-t)
avg1=sum1/len(dataset)
avg2=sum2/len(dataset)

print(f'Accuracy of the custom algorithm: {avg1}')
print(f'Accuracy of the traditional zero shot prediction: {avg2}')

**Classifying without descriptors**

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

tokenized_text = torch.cat([clip.tokenize(label) for label in labels]).to(device)
logits=model(images,tokenized_text)[0]
pred=torch.argmax(logits,dim=1)
print(f"Accuracy: {torch.mean((pred==torch.Tensor([label for (image,label) in dataset2]))+0.)}")